# R Hartmann6 with Parameter Constraints

This notebook is an example of a single objective R model optimization using a Hartmann6 function utilizing parameter constraints as well as an additional simulated black box constraint. Here our parameter constraints specify that some linear combination of some of the parameters needs to be under some constant (i.e. a budget constraint). Our simulated black box constraint is any function that we could theoretically calculate in our run_model.R script as an additional constraint (in this case we just use the l2 norm of the parameters).

The Hartmann6 function is a synthetic function that is often used as a stand in for a model that is expensive to evaluate. The function has 6 dimensions and is defined on the unit hypercube.

See the folder `r_hartmann` for the R script and the configuration file. 

In [2]:
# This notebook uses hidden cells to import and display things so
# that it can be ran regullary to make sure the documentation
# is up to date and not broken (as opposed to a markdown file
# that would have an exmaple written once and get out of date
# as the code base changes)

# These hidden cells are only responsible for rerunning
# the documentation to ensure it is correct.
# the actual relavent part of the documentation
# are the non hidden parts

In [10]:
import pathlib

from IPython.display import Code

boa_paper_dir = pathlib.Path().resolve().parent
r_dir = boa_paper_dir / "r_hartmann"
config_path = r_dir / "config_constraints.yaml"
run_model_script = r_dir / "run_model_w_l2norm.R"
hartmann6_script = r_dir / "hartmann6.R"

## Configuration File

config_constraints.yaml

In [11]:
Code(config_path)

objective:
    metrics:
        -   name: Hartmann6
    outcome_constraints:
        - l2norm <= 1.25
parameters:
    x1:
        type: range
        bounds: [0.0, 1.0]
    x2:
        type: range
        bounds: [0.0, 1.0]
    x3:
        type: range
        bounds: [0.0, 1.0]
    x4:
        type: range
        bounds: [0.0, 1.0]
    x5:
        type: range
        bounds: [0.0, 1.0]
    x6:
        type: range
        bounds: [0.0, 1.0]
parameter_constraints:
    - .5*x1 + x2 <= 1.5
n_trials: 10
script_options:
    run_model: Rscript run_model_w_l2norm.R
# generation_strategy:
#     steps:
#         -   model: SOBOL
#             num_trials: 10
#         -   model: GPEI
#             num_trials: -1

## Run Model Wrapper Script

run_model.R

In [12]:
Code(run_model_script)

# load in any libraries and modules we need
library(jsonlite)
source("./hartmann6.R")

trial_dir <- commandArgs(trailingOnly=TRUE)[1]
data <- read_json(path=file.path(trial_dir, "parameters.json"))
res <- hartmann6(unlist(data))
write(toJSON(list(Hartmann6=res, l2norm=sum(unlist(data)**2)**(1/2))), file.path(trial_dir, "output.json"))

We also use a function called hartman6 which is a 6 dimensional version of the synthetic hartman function as the stand in for our model function. The code is below. You would substitute this for any call your model, be it local call to your own R model, a system call to a fortran model wrapped in your R script, or perhaps a some code that launches an HPC job and collects the results.

hartmann6.R

In [6]:
Code(hartmann6_script)

hartmann6 <- function(X) {
     out <- tryCatch(
     {
          alpha <- c(1.0, 1.2, 3.0, 3.2)
          A <- c(10, 3, 17, 3.5, 1.7, 8,
                 0.05, 10, 17, 0.1, 8, 14,
                 3, 3.5, 1.7, 10, 17, 8,
                 17, 8, 0.05, 10, 0.1, 14)
          A <- matrix(A, 4, 6, byrow=TRUE)
          P <- 10^(-4) * c(1312, 1696, 5569, 124, 8283, 5886,
                           2329, 4135, 8307, 3736, 1004, 9991,
                           2348, 1451, 3522, 2883, 3047, 6650,
                           4047, 8828, 8732, 5743, 1091, 381)
          P <- matrix(P, 4, 6, byrow=TRUE)

          Xmat <- matrix(rep(X,times=4), 4, 6, byrow=TRUE)
          inner_sum <- rowSums(A[,1:6]*(Xmat-P[,1:6])^2)
          outer_sum <- sum(alpha * exp(-inner_sum))
          y <- -outer_sum
          return(y)
     },
       error=function(cond) {
            return(NA)
        }
     )
    return(out)
}

## Running our script

To run our script we just need to pass the config file to BOA's CLI

```python
python -m boa --config-file path/to/config.yaml
```

or

```python
python -m boa -c path/to/config.yaml
```

In [7]:
output = !python -m boa -c {config_path}  # we capture ipython terminal output to python variable
o = "\n".join(ln for ln in output)  # it comes in as a list, we convert to string
o = o.replace(str(r_dir),  "[/path/to/your/dir/]")  # replace the actual dir with a stand in for privacy reasons
Code(o)

[WARNING 07-13 14:20:39] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.
[INFO 07-13 14:20:40] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 14:20:40] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 14:20:40] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x3. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 14:20:40] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x4. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 14:20:40] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x5. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 14:20:40] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x6. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-13 14:20:40] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x1', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x3', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x4', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x5', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x6', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[ParameterConstraint(0.5*x1 + 1.0*x2 <= 1.5)]).
[INFO 07-13 14:20:40] ax.service.utils.instantiation: Due to non-specification, we will use the heuristic for selecting objective thresholds.
[INFO 07-13 14:20:40] ax.modelbridge.dispatch_utils: Using Models.BOTORCH_MODULAR since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 07-13 14:20:40] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunable_parameters=6 num_trials=None use_batch_trials=False
[INFO 07-13 14:20:40] ax.modelbridge.dispatch_utils: calculated num_initialization_trials=12
[INFO 07-13 14:20:40] ax.modelbridge.dispatch_utils: num_completed_initialization_trials=0 num_remaining_initialization_trials=12
[INFO 07-13 14:20:40] ax.modelbridge.dispatch_utils: `verbose`, `disable_progbar`, and `jit_compile` are not yet supported when using `choose_generation_strategy` with ModularBoTorchModel, dropping these arguments.
[INFO 07-13 14:20:40] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+BoTorch', steps=[Sobol for 12 trials, BoTorch for subsequent trials]). Iterations after 12 will take longer to generate due to model-fitting.
[WARNING 2024-07-13 14:20:40,532 MainProcess MainThread {ax_instantiation_utils.py:120}] boa: Multi Objective Optimization was specified,
but generation steps used step: GenerationStep(model=<Models.BOTORCH_MODULAR: 'BoTorch'>, num_trials=-1, model_kwargs={'torch_device': None, 'transform_configs': {'Derelativize': {'use_raw_status_quo